<a href="https://colab.research.google.com/github/NandithaSurisetti/Movie-reviews--Sentiment-analysis/blob/main/Generativemodels_FlanT5%2CChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers sentence_transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

data=load_dataset("rotten_tomatoes")
data


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
from transformers import pipeline

pipe=pipeline(
    "text2text-generation" ,
    model="google/flan-t5-large" ,
    device= "cuda:0"
)

Device set to use cuda:0


In [ ]:
prompt="Is the following sentence postive or negative"

data=data.map(lambda example : {"t5" : prompt + example['text']})
data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred=[]
for output in tqdm(pipe(KeyDataset(data["test"],"t5")) , total=len(data["test"])):
  text=output[0]["generated_text"]
  y_pred.append(0 if text=="negative" else 1)


100%|██████████| 1066/1066 [01:54<00:00,  9.33it/s]


In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true,y_pred):
  performance=classification_report(
      y_true,y_pred ,
      target_names=["Negative Review","Positive review"]
  )
  print (performance)

In [ ]:
evaluate_performance(data["test"]["label"] , y_pred)

                 precision    recall  f1-score   support

Negative Review       0.92      0.89      0.90       533
Positive review       0.90      0.92      0.91       533

       accuracy                           0.91      1066
      macro avg       0.91      0.91      0.91      1066
   weighted avg       0.91      0.91      0.91      1066



ChatGPT


In [3]:
!pip install datasets openai

In [4]:
from datasets import load_dataset

data=load_dataset("rotten_tomatoes")
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
import openai

client=openai.OpenAI(api_key="")

In [6]:
def chatgpt_generate(prompt, document , model="gpt-3.5-turbo-0125"):
  """Generate an output based on a prompt and an input document"""
  messages=[
      {
          "role":"system",
          "content":"You are a helpful assistant"
      },
      {
          "role" : "user" ,
          "content" : prompt.replace("[DOCUMENT]" ,document)
      }
  ]
  chat_completion=client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0
  )
  return chat_completion.choices[0].message.content



In [7]:
prompt="""Predict whether the following document is a positive or negative moview review:
[DOCUMENT]
If its positive return 1 and if it is negative return 0 .Do not give any other answer
 """
document="unpretencious,quirky,charming,original"
chatgpt_generate(prompt,document)

'1'

In [8]:
import numpy as np
from tqdm import tqdm

prediction=[chatgpt_generate(prompt,doc) for doc in tqdm(data['test']['text'])]

100%|██████████| 1066/1066 [06:50<00:00,  2.60it/s]


In [9]:
y_pred=[int(pred) for pred in prediction]

In [10]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true,y_pred):
  performance=classification_report(
      y_true,y_pred ,
      target_names=["Negative Review","Positive review"]
  )
  print (performance)

In [11]:
evaluate_performance(data['test']['label'], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.87      0.96      0.91       533
Positive review       0.96      0.86      0.90       533

       accuracy                           0.91      1066
      macro avg       0.91      0.91      0.91      1066
   weighted avg       0.91      0.91      0.91      1066

